In [6]:
from music21 import converter, instrument, note, chord
# import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import mido
import glob, pickle
import sys
import numpy as np
import os
import sklearn
import matplotlib.pyplot as plt

In [16]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    # n_vocab = len(set(np.ndarray.flatten(np.array(notes))))
    unpacked_notes = []
    
    for item in notes:
        unpacked_notes.extend(item)
    
    n_vocab = len(set(unpacked_notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)
    
    np.save("a", network_input)
    np.save("b", network_output)

    return train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        notes_i = []

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes_i.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes_i.append(str(element.pitches[-1])) # take the note with the highest octave? This is a modification
        
        # trim out the excess to standardize the length of the musical piece
        desired_length = len(notes_i) - (len(notes_i) % 50) # 50 will be our input length
        notes.append(notes_i[0:desired_length])

    assert len(notes) == len(glob.glob("midi_songs/*.mid"))
    
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 50
    
    unpacked_notes = []
    
    for item in notes:
        unpacked_notes.extend(item)

    # get all pitch names
    pitchnames = sorted(set(item for item in unpacked_notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, 1):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])
    
    assert len(network_input) == len(network_output), len(network_input)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    #model.add(Dropout(0.3))
    model.add(LSTM(n_vocab))
    #model.add(Dropout(0.3))
    #model.add(LSTM(512))
    #model.add(Dense(256))
    #model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    rmsprop = optimizers.RMSprop(lr=0.002, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history_object = model.fit(network_input, network_output, 
                               epochs=200, batch_size=64,
                               validation_split=0.2,
                               callbacks=None)
    return history_object

In [3]:
train_network()

Parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/DOS.mid
Parsing midi_songs/FF3_Battle_(Piano).mid
Parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/ff7themep.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/Cids.mid
Parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing midi_songs/AT.mid
Parsing midi_songs/8.mid
Parsing midi_songs/sobf.mid
Parsing midi_songs/redwings.mid
Parsing midi_songs/roseofmay-piano.mid
Parsing midi_songs/ff4_piano_collections-main_theme.mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/dayafter.mid
Parsing midi_songs/FFIX_Piano.mid
Parsing midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/BlueStone_LastDungeon.mid
Parsing midi

33500/33500 [==============================] - 86s 3ms/sample - loss: 0.5435 - accuracy: 0.8591
Epoch 60/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.5222 - accuracy: 0.8639
Epoch 61/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.4984 - accuracy: 0.8711
Epoch 62/200
33500/33500 [==============================] - 87s 3ms/sample - loss: 0.4814 - accuracy: 0.8765
Epoch 63/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.4645 - accuracy: 0.8805
Epoch 64/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.4447 - accuracy: 0.8861
Epoch 65/200
33500/33500 [==============================] - 83s 2ms/sample - loss: 0.4275 - accuracy: 0.8916
Epoch 66/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.4098 - accuracy: 0.8972
Epoch 67/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.3940 - accuracy: 0.9017
Epoch 68/200
33500/33500 [======

Epoch 134/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.1140 - accuracy: 0.9721
Epoch 135/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.1155 - accuracy: 0.9717
Epoch 136/200
33500/33500 [==============================] - 86s 3ms/sample - loss: 0.1151 - accuracy: 0.9716
Epoch 137/200
33500/33500 [==============================] - 86s 3ms/sample - loss: 0.1143 - accuracy: 0.9717
Epoch 138/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.1144 - accuracy: 0.9705
Epoch 139/200
33500/33500 [==============================] - 86s 3ms/sample - loss: 0.1118 - accuracy: 0.9708
Epoch 140/200
33500/33500 [==============================] - 86s 3ms/sample - loss: 0.1073 - accuracy: 0.9729
Epoch 141/200
33500/33500 [==============================] - 86s 3ms/sample - loss: 0.1127 - accuracy: 0.9715
Epoch 142/200
33500/33500 [==============================] - 85s 3ms/sample - loss: 0.1091 - accuracy: 0.9715
Epoch 143/

In [16]:
input_array = np.load("a.npy")
output = np.load("b.npy")

In [33]:
    model = Sequential()
    model.add(LSTM(
        256,1
        input_shape=(input_array.shape[1], input_array.shape[2]),
        return_sequences=True
    ))
    #model.add(Dropout(0.3))
    model.add(LSTM(77))
    #model.add(Dropout(0.3))
    #model.add(LSTM(512))
    #model.add(Dense(256))
    #model.add(Dropout(0.3))
    model.add(Dense(77))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_16 (UnifiedLSTM (None, 50, 256)           264192    
_________________________________________________________________
unified_lstm_17 (UnifiedLSTM (None, 77)                102872    
_________________________________________________________________
dense_11 (Dense)             (None, 77)                6006      
_________________________________________________________________
activation_11 (Activation)   (None, 77)                0         
Total params: 373,070
Trainable params: 373,070
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
lr_2 = train_network()

Parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/DOS.mid
Parsing midi_songs/FF3_Battle_(Piano).mid
Parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/ff7themep.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/Cids.mid
Parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing midi_songs/AT.mid
Parsing midi_songs/8.mid
Parsing midi_songs/sobf.mid
Parsing midi_songs/redwings.mid
Parsing midi_songs/roseofmay-piano.mid
Parsing midi_songs/ff4_piano_collections-main_theme.mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/dayafter.mid
Parsing midi_songs/FFIX_Piano.mid
Parsing midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/BlueStone_LastDungeon.mid
Parsing midi